# JBArm.py test

In [1]:
# test JBArm
import numpy as np
from JBArm import JBArm
from time import sleep

arm = JBArm()

# move along with X axis
arm.move_to_xyz(arm.x + np.array([50, 0, 0]))
sleep(2)

# move along with Z axis
arm.move_to_xyz(arm.x + np.array([0, 0, -100]))
sleep(2)

# move along with Y axis
arm.move_to_xyz(np.array([150, -150, 150]))
sleep(2)

arm.move_to_xyz(arm.x + np.array([0, 300, 0]))
sleep(2)

# to ready position
arm.ready()

# move along with X, Z plain
arm.move_to_xyz(arm.x + np.array([50, 0, -230]))
sleep(2)

# grab
arm.grab()

# release
arm.release()

# to ready position
arm.ready()

Succeeded to open the port
Succeeded to change the baudrate


# Aruco Maker Visualization

In [6]:
# widget
import traitlets
import ipywidgets
from IPython.display import display

# JetBot 카메라
from SCSCtrl import TTLServo
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()
TTLServo.servoAngleCtrl(1, -7, 1, 150)
TTLServo.servoAngleCtrl(5, -30, 1, 150)

371

In [10]:
# essential
import time
import numpy as np
import cv2

# widget
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# user
from BoxDetector import BoxDetector

box_detector = BoxDetector()

def tf(frame):
    fr = frame.copy()
    fr = cv2.resize(fr, (300, 300), interpolation=cv2.INTER_LINEAR)
    
    boxes = box_detector.detect_boxes(fr)
    
    for num, x_box in boxes.items():
        x_cam = np.linalg.inv(box_detector.A) @ np.hstack([x_box, 1])
        x_cam = x_cam[:3] / x_cam[2]
        x_img = box_detector.cam_intrinsic @ x_cam
        
        x, y = int(x_img[0]), int(x_img[1])
        
        cv2.circle(fr, (x, y), 4, (255, 0, 0), -1)
        cv2.putText(fr, f"ID: {num}", (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 0), 1)
    
#     time.sleep(0.05)
    return bgr8_to_jpeg(fr)
    
img_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera
cam_link = traitlets.dlink((camera, 'value'), (img_widget, 'value'), transform=tf)
display(img_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
camera.stop()

# BoxDetector and JBArm test

In [11]:
# essential
from time import sleep
import numpy as np

# user
from BoxDetector import BoxDetector
from JBArm import JBArm

arm = JBArm()
box_detector = BoxDetector()

In [15]:
# marker info
fr = cv2.resize(camera.value, (300, 300), interpolation=cv2.INTER_LINEAR)
boxes = box_detector.detect_boxes(fr)
for num, x_box in boxes.items():
    print(f"ID: {num} at {x_box}")

box_idx = 0
x_box = boxes[box_idx]

arm.pick(x_box)
arm.place(x_box + np.array([0, -100, 0]))

ID: 0 at [154.67586952  42.18133951 -71.8633672 ]


# Collect Dataset

In [ ]:
import ipywidgets.widgets as widgets

robot = Robot()
robot.set_motors(0.5, 0.5)

x_slider = widgets.FloatSlider(min=-0.5, max=0.5, step=0.001, description='x')

def on_change_x(change):
    r_speed = 0.5 + x_slider.value
    l_speed = 1 - r_speed
    robot.set_motors(l_speed, r_speed)
    
x_slider.observe(on_change_x, names='value')

display(widgets.VBox([x_slider]))

In [ ]:
robot.stop()